In [3]:
LOG_PATH = r"D:\code\WindowsAgentArena\src\win-arena-container\client\results\pyautogui\a11y_tree\gpt-4-vision-preview\0"

In [4]:
import os
import pandas as pd

final_result = []
total_tests = 0
successful_tests = 0

app_domains = os.listdir(LOG_PATH)
for app in app_domains:
    app_domain_path = os.path.join(LOG_PATH, app)
    log_folders = os.listdir(app_domain_path)

    app_success_num = 0
    for folder in log_folders:
        result_log_path = os.path.join(app_domain_path, folder, "result.txt")
        try:
            with open(result_log_path, "r", encoding="utf-8") as f:
                result = str(f.read()).strip()
            if result == "1" or result == "1.0" or float(result) > 0.7:
                app_success_num += 1
                print(f"{app} - {folder}: {result}")
        except Exception:
            pass

    template_format = {
        " ": app,
        "Total Tests": len(log_folders),
        "Successful Tests": app_success_num,
        "Success Rate": app_success_num / len(log_folders)
    }
    successful_tests += app_success_num
    total_tests += len(log_folders)

    final_result.append(template_format)

final_result.append(
    {
        " ": "OVERALL",
        "Total Tests": total_tests,
        "Successful Tests": successful_tests,
        "Success Rate": successful_tests / total_tests,
    }
)

for item in final_result:
    print(item)
# df = pd.DataFrame(final_result)
# excel_path = r"D:\code\WindowsAgentArena\src\win-arena-container\client\results\view.xlsx"
# df.to_excel(excel_path, index=False, engine="openpyxl")

multi_app - 6d72aad6-187a-4392-a4c4-ed87269c51cf: 1
multi_app - a82b78bb-7fde-4cb3-94a4-035baf10bcf0: 1
word - 0b17a146-2934-46c7-8727-73ff6b6483e8: 1
word - 0e763496-b6bb-4508-a427-fad0b6c3e195: 1
word - ecc2413d-8a48-416e-a3a2-d30106ca36cb: 1
{' ': 'excel', 'Total Tests': 11, 'Successful Tests': 0, 'Success Rate': 0.0}
{' ': 'multi_app', 'Total Tests': 22, 'Successful Tests': 2, 'Success Rate': 0.09090909090909091}
{' ': 'ppt', 'Total Tests': 7, 'Successful Tests': 0, 'Success Rate': 0.0}
{' ': 'word', 'Total Tests': 9, 'Successful Tests': 3, 'Success Rate': 0.3333333333333333}
{' ': 'OVERALL', 'Total Tests': 49, 'Successful Tests': 5, 'Success Rate': 0.10204081632653061}


In [2]:
from typing import Iterable
import csv

def debug_csv_difference(file1: str, file2: str, encoding='utf-8-sig', delimiter=','):
    with open(file1, encoding="utf-8-sig") as f:
        result_lines: Iterable[str] = f.read().splitlines()
    with open(file2, encoding="utf-8-sig") as f:
        expected_lines: Iterable[str] = f.read().splitlines()

    # 检查行数
    len_r, len_e = len(result_lines), len(expected_lines)
    if len_r != len_e:
        print(f"⚠️ 行数不一致：result={len_r} 行, expected={len_e} 行")

    # 按行比对
    match = True
    for i, (r_line, e_line) in enumerate(zip(result_lines, expected_lines), start=1):
        if r_line != e_line:
            match = False
            print(f"\n🔍 第 {i} 行不一致:")
            print(f"  result  : {repr(r_line)}")
            print(f"  expected: {repr(e_line)}")

    # 检查剩余多出的行
    if len_r > len_e:
        match = False
        for i in range(len_e, len_r):
            print(f"\n📌 result 多出的第 {i+1} 行: {repr(result_lines[i])}")
    elif len_e > len_r:
        match = False
        for i in range(len_r, len_e):
            print(f"\n📌 expected 多出的第 {i+1} 行: {repr(expected_lines[i])}")

    metric: bool = list(result_lines) == list(expected_lines)
    print(metric)

debug_csv_difference(
    file1=r"D:\code\WindowsAgentArena\src\win-arena-container\client\cache\3aaa4e37-dc91-482e-99af-132a612d40f3\Export_Calc_to_CSV.csv",
    file2=r"D:\code\WindowsAgentArena\src\win-arena-container\client\cache\3aaa4e37-dc91-482e-99af-132a612d40f3\Export_Calc_to_CSV_gold.csv"
)

True


In [11]:
import pandas as pd
import functools
from typing import Union, List, Tuple
from openpyxl import load_workbook
from pandas import ExcelFile, DataFrame


def _parse_sheet_idx(sheet_idx: Union[str, int],
                     pdworkbookr: ExcelFile,
                     pdworkbooke: ExcelFile = None,
                     result_sheet_names: List[str] = None,
                     expected_sheet_names: List[str] = None
                     ) -> Tuple[ExcelFile, str]:
    """
    将 sheet 索引（数字或字符串）转换为 (workbook对象, sheet名称)
    支持格式:
    - int: 表示 result 的第几个 sheet
    - "RI0": result index 0
    - "EI0": expected index 0
    - "RNsheet": result by name
    - "ENsheet": expected by name
    """
    if isinstance(sheet_idx, int):
        return pdworkbookr, result_sheet_names[sheet_idx]

    if isinstance(sheet_idx, str):
        if sheet_idx.startswith("RI"):
            idx = int(sheet_idx[2:])
            return pdworkbookr, result_sheet_names[idx]
        elif sheet_idx.startswith("EI"):
            idx = int(sheet_idx[2:])
            return pdworkbooke, expected_sheet_names[idx]
        elif sheet_idx.startswith("RN"):
            name = sheet_idx[2:]
            return pdworkbookr, name
        elif sheet_idx.startswith("EN"):
            name = sheet_idx[2:]
            return pdworkbooke, name

    raise ValueError(f"无法识别的 sheet_idx: {sheet_idx}")


def _load_sheet(workbook: ExcelFile, sheet_name: str) -> Union[DataFrame, None]:
    """加载 sheet 为 pandas.DataFrame，失败则返回 None"""
    try:
        return workbook.parse(sheet_name=sheet_name)
    except Exception as e:
        print(f"❌ 加载 sheet '{sheet_name}' 失败: {e}")
        return None


def debug_compare_table(result: str, expected: str = None, **options) -> float:
    if result is None:
        print("❌ 未提供 result 文件路径")
        return 0.

    try:
        xlworkbookr = load_workbook(filename=result)
        pdworkbookr = pd.ExcelFile(result)
    except Exception as e:
        print(f"❌ 加载 result 失败: {e}")
        return 0.

    if expected:
        try:
            xlworkbooke = load_workbook(filename=expected)
            pdworkbooke = pd.ExcelFile(expected)
        except Exception as e:
            print(f"❌ 加载 expected 失败: {e}")
            return 0.
    else:
        xlworkbooke = pdworkbooke = None

    worksheetr_names = pdworkbookr.sheet_names
    worksheete_names = pdworkbooke.sheet_names if expected else None

    parse_idx = functools.partial(
        _parse_sheet_idx,
        result_sheet_names=worksheetr_names,
        expected_sheet_names=worksheete_names
    )

    passes = True

    for r in options["rules"]:
        if r["type"] == "sheet_data":
            sheet1 = _load_sheet(*parse_idx(r["sheet_idx0"], pdworkbookr, pdworkbooke))
            sheet2 = _load_sheet(*parse_idx(r["sheet_idx1"], pdworkbookr, pdworkbooke))

            if sheet1 is None or sheet2 is None:
                print("❌ 表格加载失败")
                return 0.

            error_limit = r.get("precision", 4)
            sheet1 = sheet1.round(error_limit)
            sheet2 = sheet2.round(error_limit)

            print(f"\n🔍 比对表格: {r['sheet_idx0']} vs {r['sheet_idx1']}")
            print(f"📐 表1 shape: {sheet1.shape}, 表2 shape: {sheet2.shape}")

            if sheet1.shape != sheet2.shape:
                print("⚠️ 表格大小不一致！")
                passes = False

            # 重设索引对齐再比较
            sheet1 = sheet1.reset_index(drop=True)
            sheet2 = sheet2.reset_index(drop=True)

            if not sheet1.equals(sheet2):
                passes = False
                print("❌ 表格内容不一致！打印差异：")

                diff_mask = sheet1 != sheet2
                for row_idx, row in diff_mask.iterrows():
                    for col in diff_mask.columns[row.values]:
                        val1 = sheet1.at[row_idx, col]
                        val2 = sheet2.at[row_idx, col]
                        print(f"  ❗ 第 {row_idx+1} 行, 列 '{col}' 不同: {repr(val1)} ≠ {repr(val2)}")
            else:
                print("✅ 表格内容一致")

        else:
            print(f"⚠️ 暂不支持类型 '{r['type']}' 的 debug")
            continue

        if not passes:
            break

    return float(passes)

debug_compare_table(
    result=r"D:\code\WindowsAgentArena\src\win-arena-container\client\cache\51b11269-2ca8-4b2a-9163-f21758420e78\Arrang_Value_min_to_max.xlsx",
    expected=r"D:\code\WindowsAgentArena\src\win-arena-container\client\cache\51b11269-2ca8-4b2a-9163-f21758420e78\Arrang_Value_min_to_max_gold.xlsx",
    rules=[
        {
            "type": "sheet_data",
            "sheet_idx0": 0,
            "sheet_idx1": "EI0"
        }
    ]

)


🔍 比对表格: 0 vs EI0
📐 表1 shape: (18, 4), 表2 shape: (18, 4)
❌ 表格内容不一致！打印差异：
  ❗ 第 1 行, 列 'Date' 不同: Timestamp('2022-11-28 00:00:00') ≠ 44893
  ❗ 第 2 行, 列 'Date' 不同: Timestamp('2023-01-06 00:00:00') ≠ 44932
  ❗ 第 3 行, 列 'Date' 不同: Timestamp('2022-12-07 00:00:00') ≠ 44902
  ❗ 第 4 行, 列 'Date' 不同: Timestamp('2023-01-17 00:00:00') ≠ 44943
  ❗ 第 5 行, 列 'Date' 不同: Timestamp('2023-01-20 00:00:00') ≠ 44946
  ❗ 第 6 行, 列 'Date' 不同: Timestamp('2022-11-19 00:00:00') ≠ 44884
  ❗ 第 7 行, 列 'Date' 不同: Timestamp('2023-01-22 00:00:00') ≠ 44948
  ❗ 第 8 行, 列 'Date' 不同: Timestamp('2023-02-04 00:00:00') ≠ 44961
  ❗ 第 9 行, 列 'Date' 不同: Timestamp('2022-12-22 00:00:00') ≠ 44917
  ❗ 第 10 行, 列 'Date' 不同: Timestamp('2022-12-01 00:00:00') ≠ 44896
  ❗ 第 11 行, 列 'Date' 不同: Timestamp('2022-12-07 00:00:00') ≠ 44902
  ❗ 第 12 行, 列 'Date' 不同: Timestamp('2023-02-03 00:00:00') ≠ 44960
  ❗ 第 13 行, 列 'Date' 不同: Timestamp('2022-12-01 00:00:00') ≠ 44896
  ❗ 第 14 行, 列 'Date' 不同: Timestamp('2022-12-16 00:00:00') ≠ 44911
  ❗ 第 15 行, 

0.0